In [57]:
import time
import requests
import os
from snownlp import SnowNLP
import numpy as np

In [58]:


def get_data(data):
    data_list = []
    comment_data_list = data["data"]["replies"]
    for i in comment_data_list:
        data_list.append((i['rpid'],
                          i['like'],
                          i['member']['uname'],
                          i['member']['sex'],
                          i['member']['level_info']['current_level'],
                          i['content']['message'],
                          i['reply_control']['time_desc']))
    return data_list


def save_data(data_type, data):
    if not os.path.exists(data_type + r'_data.csv'):
        f = open(data_type + r"_data.csv", "w", encoding='utf-8')
        f.write("rpid,点赞数量,用户,性别,等级,评论内容,发布时间\n")
    else:
        f = open(data_type + r"_data.csv", "a+", encoding='utf-8')
    for comment_info in data:
        rpid,like_count,user,sex,level,content,time = comment_info
        user = user.replace(',', '，')
        content = content.replace(',', '，')
        content = content.replace('\n','')
        content = content[(content.find(":")+1):]
        row = '{},{},{},{},{},{},{}'.format(rpid,like_count,user,sex,level,content,time)
        f.write(row)
        f.write('\n')
    f.close()

def bv2av(bv:str)->str:
    response = requests.get(url='https://api.bilibili.com/x/web-interface/view',params={'bvid':bv})
    av = str(response.json()['data']['aid'])
    return av


In [60]:
!rm *.csv
video_url =  'https://www.bilibili.com/video/BV1Dh41167W4'
bv = video_url.split('/')[-1]
oid = bv2av(bv)
for i in range(30000):
    url = "https://api.bilibili.com/x/v2/reply/main?jsonp=jsonp&next={}&type=1&oid={}&mode=3&plat=1".format(str(i),oid)
    print(url)
    d = requests.get(url)
    data = d.json()
    if not data['data']['replies']:
        break
    m_data = get_data(data)
    save_data("main", m_data)
    for j in m_data:
        reply_url = "https://api.bilibili.com/x/v2/reply/reply?jsonp=jsonp&pn=1&type=1&oid={}&ps=10&root={}".format(oid,str(j[0]))
        r = requests.get(reply_url)
        r_data = r.json()
        if not r_data['data']['replies']:
            break
        reply_data = get_data(r_data)
        save_data("reply", reply_data)
        # break
        time.sleep(5)
    # break
    time.sleep(5)


https://api.bilibili.com/x/v2/reply/main?jsonp=jsonp&next=0&type=1&oid=206719735&mode=3&plat=1
https://api.bilibili.com/x/v2/reply/main?jsonp=jsonp&next=1&type=1&oid=206719735&mode=3&plat=1
https://api.bilibili.com/x/v2/reply/main?jsonp=jsonp&next=2&type=1&oid=206719735&mode=3&plat=1
https://api.bilibili.com/x/v2/reply/main?jsonp=jsonp&next=3&type=1&oid=206719735&mode=3&plat=1
